In [ ]:
!pip install pydub

In [ ]:
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [ ]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 17.9 MB/s eta 0:00:00


In [ ]:
import os

# Paths
video_file_path = '/content/sample_data/video.mp4'  # Replace with your video file path
audio_file_path = '/content/sample_data/audio/audio.wav'  # Save extracted audio as WAV
muted_video_path = '/content/sample_data/muted_video.mp4'  # Save muted video

# Extract audio from video
os.system(f'ffmpeg -i "{video_file_path}" -q:a 0 -map a "{audio_file_path}"')
print(f"Audio extracted and saved at: {audio_file_path}")

# Create muted video (video without audio)
os.system(f'ffmpeg -i "{video_file_path}" -c copy -an "{muted_video_path}"')
print(f"Muted video saved at: {muted_video_path}")


Audio extracted and saved at: /content/sample_data/audio/audio.wav
Muted video saved at: /content/sample_data/muted_video.mp4


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler

# Inputs
model_folder = "/content/sample_data/model"
video_path = "/content/sample_data/video.mp4"

# Load model and scaler
model = keras.models.load_model(os.path.join(model_folder, "dnn_model.h5"),
                                custom_objects={"mse": keras.losses.MeanSquaredError()})
scaler_mean = np.load(os.path.join(model_folder, "scaler.npy"))
scaler_std = np.load(os.path.join(model_folder, "scaler_std.npy"))
scaler = StandardScaler()
scaler.mean_ = scaler_mean
scaler.scale_ = scaler_std

# Feature extraction function
def extract_features(frame, frame_index):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
    contrast = np.std(gray)
    hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
    hist /= hist.sum()
    entropy = -np.sum(hist * np.log2(hist + 1e-10))
    edges = cv2.Canny(gray, 50, 150)
    edge_density = np.sum(edges) / (frame.shape[0] * frame.shape[1])
    pixels = frame.reshape((-1, 3)).astype(np.float32)
    _, labels, _ = cv2.kmeans(pixels, 2, None,
                              (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0),
                              10, cv2.KMEANS_RANDOM_CENTERS)
    color_ratio = np.max(np.bincount(labels.flatten())) / len(labels)
    dct = cv2.dct(np.float32(gray) / 255.0)
    dct_energy = np.sum(np.abs(dct[:10, :10]))
    frame_diff = np.abs(cv2.absdiff(gray, cv2.blur(gray, (5, 5)))).mean()

    return [frame_index, laplacian_var, contrast, entropy, edge_density, color_ratio, dct_energy, frame_diff]

# Process video
cap = cv2.VideoCapture(video_path)
frame_index = 0
features_list = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    if frame_index % 5 == 0:
        features = extract_features(frame, frame_index)
        if features:
            features_list.append(features)
    frame_index += 1
cap.release()

if not features_list:
    print("❌ No valid frames extracted!")
    exit()

# Convert to DataFrame
df = pd.DataFrame(features_list, columns=["Frame Index", "Blurriness", "Contrast", "Entropy", "Edge Density", "Color Ratio", "DCT Energy", "Frame Difference"])
X_test = scaler.transform(df.iloc[:, 1:].values)

# Predict using the trained model
df["Predicted Unsuitability Score"] = model.predict(X_test)

# Save predictions
csv_path = "/content/sample_data/InfoOfSecretText/predictions.csv"
df.to_csv(csv_path, index=False)
print(f"📄 Saved predictions: {csv_path}")

# Select 5 most unsuitable frames
selected_frames = df.nlargest(5, "Predicted Unsuitability Score")["Frame Index"].tolist()
frame_indices_str = ",".join(map(str, selected_frames))

# Save frame indices to metadata file
metadata_path = "/content/sample_data/InfoOfSecretText/metadata.txt"
with open(metadata_path, "w") as f:
    f.write(frame_indices_str)

print(f"✅ Processed video. Frame indices saved in '{metadata_path}'")
# Ensure the directory for frames exists
frames_dir = "/content/sample_data/frames_folder"
os.makedirs(frames_dir, exist_ok=True)

# Save selected frames as images
cap = cv2.VideoCapture(video_path)
for frame_idx in selected_frames:
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
    ret, frame = cap.read()
    if ret:
        frame_filename = os.path.join(frames_dir, f"frame_{frame_idx}.png")
        cv2.imwrite(frame_filename, frame)
cap.release()

print(f"🖼️ Frames saved in '{frames_dir}'")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step 
📄 Saved predictions: /content/sample_data/InfoOfSecretText/predictions.csv
✅ Processed video. Frame indices saved in '/content/sample_data/InfoOfSecretText/metadata.txt'
🖼️ Frames saved in '/content/sample_data/frames_folder'


In [ ]:
import os
import base64
import hashlib
import json
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Util.Padding import pad, unpad

# Define directories
WORKING_DIR = "/content/sample_data/"
KEYS_DIR = os.path.join(WORKING_DIR, "keys/")
ENCRYPTED_DATA_DIR = os.path.join(WORKING_DIR, "encrypted_data/")

# Ensure directories exist
os.makedirs(KEYS_DIR, exist_ok=True)
os.makedirs(ENCRYPTED_DATA_DIR, exist_ok=True)

# **1. Generate AES Key (256-bit)**
aes_key = os.urandom(32)  # 256-bit key
aes_key_bin_path = os.path.join(KEYS_DIR, "aes_key.bin")
aes_key_txt_path = os.path.join(KEYS_DIR, "aes_key.txt")

# Save AES key in binary
with open(aes_key_bin_path, "wb") as f:
    f.write(aes_key)

# Save AES key in Base64 format (text)
with open(aes_key_txt_path, "w") as f:
    f.write(base64.b64encode(aes_key).decode())

print(f"🔑 AES Key saved at: {aes_key_bin_path}")
print(f"📜 AES Key (Base64) saved at: {aes_key_txt_path}")

# **2. Generate RSA Key Pair (2048-bit)**
rsa_key = RSA.generate(2048)
private_key_path = os.path.join(KEYS_DIR, "private_key.pem")
public_key_path = os.path.join(KEYS_DIR, "public_key.pem")

with open(private_key_path, "wb") as f:
    f.write(rsa_key.export_key())
with open(public_key_path, "wb") as f:
    f.write(rsa_key.publickey().export_key())

print(f"🔐 RSA Private Key saved at: {private_key_path}")
print(f"📜 RSA Public Key saved at: {public_key_path}")

# **3. Encrypt AES Key using RSA Public Key**
public_key = RSA.import_key(open(public_key_path).read())
cipher_rsa = PKCS1_OAEP.new(public_key)
encrypted_aes_key = cipher_rsa.encrypt(aes_key)

# Save encrypted AES key in binary & text formats
encrypted_aes_key_bin_path = os.path.join(KEYS_DIR, "encrypted_aes_key.bin")
encrypted_aes_key_txt_path = os.path.join(KEYS_DIR, "encrypted_aes_key.txt")

with open(encrypted_aes_key_bin_path, "wb") as f:
    f.write(encrypted_aes_key)
with open(encrypted_aes_key_txt_path, "w") as f:
    f.write(base64.b64encode(encrypted_aes_key).decode())

print(f"🔏 AES Key encrypted and saved as binary: {encrypted_aes_key_bin_path}")
print(f"📜 AES Key encrypted and saved as text (Base64): {encrypted_aes_key_txt_path}")

# **4. Read and Encrypt Secret Text File**
secret_text_path = os.path.join(WORKING_DIR, "secret_text.txt")
with open(secret_text_path, "r") as f:
    secret_text = f.read().strip()

# Encrypt using AES-CBC
iv = os.urandom(16)
cipher_aes = AES.new(aes_key, AES.MODE_CBC, iv)
cipher_text = iv + cipher_aes.encrypt(pad(secret_text.encode(), AES.block_size))

# Save encrypted text in binary & text formats
encrypted_text_bin_path = os.path.join(ENCRYPTED_DATA_DIR, "encrypted_text.bin")
encrypted_text_txt_path = os.path.join(ENCRYPTED_DATA_DIR, "encrypted_text.txt")

with open(encrypted_text_bin_path, "wb") as f:
    f.write(cipher_text)
with open(encrypted_text_txt_path, "w") as f:
    f.write(base64.b64encode(cipher_text).decode())

print(f"📝 Secret text encrypted and saved in binary: {encrypted_text_bin_path}")
print(f"📄 Secret text encrypted and saved in text format: {encrypted_text_txt_path}")

# **5. Generate Hybrid Hash (MD5 → SHA-256)**
md5_hash = hashlib.md5(secret_text.encode()).hexdigest()
final_hash = hashlib.sha256(md5_hash.encode()).hexdigest()

hash_path = os.path.join(ENCRYPTED_DATA_DIR, "hash_value.txt")
with open(hash_path, "w") as f:
    f.write(final_hash)

print(f"🔗 Hash value saved at: {hash_path}")

# **6. Store Metadata**
metadata = {
    "original_text_size": len(secret_text),
    "aes_key_size": len(aes_key) * 8,
    "rsa_key_size": rsa_key.size_in_bits(),
    "hash": final_hash,
}

metadata_path = os.path.join(ENCRYPTED_DATA_DIR, "metadata.json")
with open(metadata_path, "w") as f:
    json.dump(metadata, f, indent=4)

print(f"📊 Metadata saved at: {metadata_path}")

# **7. Decryption Verification**
# Load private key
private_key = RSA.import_key(open(private_key_path).read())
cipher_rsa = PKCS1_OAEP.new(private_key)

# Decrypt AES Key
decrypted_aes_key = cipher_rsa.decrypt(encrypted_aes_key)
assert decrypted_aes_key == aes_key, "❌ AES Key decryption failed!"

# Decrypt Secret Text
iv = cipher_text[:16]  # Extract IV
encrypted_data = cipher_text[16:]
cipher_aes = AES.new(decrypted_aes_key, AES.MODE_CBC, iv)
decrypted_text = unpad(cipher_aes.decrypt(encrypted_data), AES.block_size).decode()

assert decrypted_text == secret_text, "❌ Decryption mismatch!"
print("✅ Encryption & Decryption verified successfully!")


🔑 AES Key saved at: /content/sample_data/keys/aes_key.bin
📜 AES Key (Base64) saved at: /content/sample_data/keys/aes_key.txt
🔐 RSA Private Key saved at: /content/sample_data/keys/private_key.pem
📜 RSA Public Key saved at: /content/sample_data/keys/public_key.pem
🔏 AES Key encrypted and saved as binary: /content/sample_data/keys/encrypted_aes_key.bin
📜 AES Key encrypted and saved as text (Base64): /content/sample_data/keys/encrypted_aes_key.txt
📝 Secret text encrypted and saved in binary: /content/sample_data/encrypted_data/encrypted_text.bin
📄 Secret text encrypted and saved in text format: /content/sample_data/encrypted_data/encrypted_text.txt
🔗 Hash value saved at: /content/sample_data/encrypted_data/hash_value.txt
📊 Metadata saved at: /content/sample_data/encrypted_data/metadata.json
✅ Encryption & Decryption verified successfully!


In [ ]:
!pip install imageio[ffmpeg] ffmpeg-python

In [ ]:
import cv2
import os
import re

# Define working directories
WORKING_DIR = "/content/sample_data/"
ENCRYPTED_TEXT_PATH = WORKING_DIR + "encrypted_data/encrypted_text.txt"
HASH_VALUE_PATH = WORKING_DIR + "encrypted_data/hash_value.txt"
IMAGE_FOLDER = WORKING_DIR + "frames_folder/"
STEGO_OUTPUT_DIR = WORKING_DIR + "stego_frames/"

# Ensure output directory exists
os.makedirs(STEGO_OUTPUT_DIR, exist_ok=True)

# Convert text to binary
def text_to_binary(text):
    return ''.join(format(ord(char), '08b') for char in text)

# Embed data in LSB of image
def embed_data(image_path, output_path, binary_message, text_part):
    img = cv2.imread(image_path)
    if img is None:
        print(f"❌ Error: Image {image_path} not found or could not be loaded!")
        return False

    height, width, channels = img.shape
    total_pixels = height * width * 3  # Total available bits for embedding

    if len(binary_message) > total_pixels:
        print(f"❌ Error: Message too large to fit in {image_path}!")
        return False

    # Flatten image and embed data
    flat_img = img.flatten()
    for i in range(len(binary_message)):
        flat_img[i] = (flat_img[i] & 0xFE) | int(binary_message[i])  # Replace LSB

    # Reshape and save image
    stego_img = flat_img.reshape((height, width, channels))
    cv2.imwrite(output_path, stego_img)

    print(f"✅ Embedded text into {output_path} (Original Frame: {os.path.basename(image_path)})")
    print(f"   → Embedded Data: \"{text_part}\"")
    print("   ----------------------------------------------------")

    return True

# Extract frame index from filename (e.g., "frame_23.png" → 23)
def extract_frame_index(filename):
    match = re.search(r"frame_(\d+)\.png", filename)
    return int(match.group(1)) if match else None

# Get the first 5 frames from the folder
def get_first_five_frames(folder):
    frame_files = [f for f in os.listdir(folder) if f.startswith("frame_") and f.endswith(".png")]
    frame_files = [(extract_frame_index(f), f) for f in frame_files if extract_frame_index(f) is not None]

    # Sort frames by index
    frame_files.sort(key=lambda x: x[0])

    if len(frame_files) < 5:
        print(f"❌ Error: Found only {len(frame_files)} valid frames! At least 5 required.")
        exit()

    return frame_files[:5]

# Split encrypted text into 4 equal parts
def split_text_into_4_parts(text):
    chunk_size = len(text) // 4
    remainder = len(text) % 4  # Handle extra characters

    split_texts = []
    start = 0
    for i in range(4):
        end = start + chunk_size + (1 if i < remainder else 0)  # Distribute remainder
        split_texts.append(text[start:end])
        start = end

    return split_texts

if __name__ == "__main__":
    # Load frames from folder
    frame_data = get_first_five_frames(IMAGE_FOLDER)
    print(f"✅ Found {len(frame_data)} frames in folder.")

    # Read encrypted text and hash value
    with open(ENCRYPTED_TEXT_PATH, "r") as f:
        encrypted_text = f.read().strip()

    with open(HASH_VALUE_PATH, "r") as f:
        hash_value_text = f.read().strip()

    # Step 1: Split encrypted text into 4 equal parts
    split_messages = split_text_into_4_parts(encrypted_text)

    print("\n🔹 **Starting Data Embedding Process** 🔹\n")

    # Step 2: Embed each text part into the first 4 images, and hash value in the last frame
    for i in range(5):
        frame_index, frame_filename = frame_data[i]
        image_path = os.path.join(IMAGE_FOLDER, frame_filename)
        output_image_path = os.path.join(STEGO_OUTPUT_DIR, f"stego_{frame_filename}")

        if i < 4:  # First 4 frames get the encrypted text parts
            text_part = split_messages[i]
        else:  # Last frame gets the hash value text
            text_part = hash_value_text

        # Convert to binary and embed
        binary_message = text_to_binary(text_part)
        embed_data(image_path, output_image_path, binary_message, text_part)

    print("\n✅ **All stego images saved successfully in:**", STEGO_OUTPUT_DIR)


✅ Found 5 frames in folder.

🔹 **Starting Data Embedding Process** 🔹

✅ Embedded text into /content/sample_data/stego_frames/stego_frame_160.png (Original Frame: frame_160.png)
   → Embedded Data: "mcooIL5xJJEBnQSWMtgXKdqHraD+WZt/pADhZo4sr0yzd9x/CwRiX0g4DjvQiwV3Cu1ZxU9IlVwGvf3MSpQ8qvTzhKZFC4o2lTyxb48sa483GRisOqE1TCAupnFYSoVKmGOazoZlM8IZ5qqBPFPbrpr4cNBolOezeV/bcNedlphyae72lV2htw3hBg2ceoBe9YWva+zbi4rhN1bPlaMs3MLjB58/OpKzx0sq9dqBNwLMNdKF7tiDKPueDQ01CT6E6GOL1aNNN1UJ/8Je8WN7MfksvB7qzfhbDiI5g/eDwbYVqOhVl9gfw0jaG8rVktyAMxcet4f/k6DFV/Llq98bsSf3/WE7v8qwBaDm0vHsZ11iCiXsC+Tn49P0iEtEApDoltrf1cEHTGxtA7YnNDpmNtc+3jNajCxVXnkj2LTkCXdo5U3578+8RzAoRSFr+UrrrZ50Hca1X1zVRCo3ZXkxWnHVEK6bLB8ao9gS+pgZKfEgJIIahzrf/ldY7LXXydu1RztR6xkIrFEI++Bju5nAABKhkfBjyMQTrtVnUS8d4NK1zPougYuYgF4WZhmMJxfc2k+NoQV4Q627inRgscDJ+Ye5uXdcwESmMbuRdv/IrYwUeAmhv7XL7rGLyyw2oBoyzQbbHcqBgdMcirgPd+p9PP4EAPHPGciKy0EhB05gOes9ISnAdf+yKDBgaMB0N9fenff6rvqoWkAy9spLhpb4A5o+netgqEdp4+TXiODRtbMv5ROi55t7puBa+K1GhGtkgWwXk0TfrZ2/5feNyYSoXnJfOdvcPSjCBN

In [ ]:
import os

# Define directories
WORKING_DIR = "/content/sample_data/"
INFO_DIR = os.path.join(WORKING_DIR, "InfoOfSecretText/")
ENCRYPTED_DATA_DIR = os.path.join(WORKING_DIR, "encrypted_data/")

# Ensure directories exist
os.makedirs(INFO_DIR, exist_ok=True)
os.makedirs(ENCRYPTED_DATA_DIR, exist_ok=True)

# Read metadata and encrypted text
metadata_path = os.path.join(INFO_DIR, "metadata.txt")
encrypted_text_path = os.path.join(ENCRYPTED_DATA_DIR, "encrypted_text.txt")

with open(metadata_path, "r") as f:
    metadata_content = f.read().strip()
with open(encrypted_text_path, "r") as f:
    encrypted_text_content = f.read().strip()

encrypted_text_length = len(encrypted_text_content)
info_file_content = f"{metadata_content}|{encrypted_text_length}"

# Save info.txt
info_file_path = os.path.join(ENCRYPTED_DATA_DIR, "info.txt")
with open(info_file_path, "w") as f:
    f.write(info_file_content)

print(f"✅ info.txt created successfully at: {info_file_path}")

✅ info.txt created successfully at: /content/sample_data/encrypted_data/info.txt


In [ ]:
import wave
import os
import random
import base64
import hashlib
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Util.Padding import pad

# Define working directory
WORKING_DIR = "/content/sample_data/"

# Paths to existing keys
AES_KEY_PATH = WORKING_DIR + "keys/aes_key.bin"
PUBLIC_KEY_PATH = WORKING_DIR + "keys/public_key.pem"
INPUT_TEXT_PATH = WORKING_DIR + "encrypted_data/info.txt"
INPUT_AUDIO_PATH = WORKING_DIR + "audio/audio.wav"
OUTPUT_AUDIO_PATH = WORKING_DIR + "audio/stego_audio.wav"

# Hybrid Hashing (MD5 → SHA-256)
def hybrid_hash(message):
    md5_hash = hashlib.md5(message.encode('utf-8')).hexdigest()
    sha256_hash = hashlib.sha256(md5_hash.encode('utf-8')).hexdigest()
    return sha256_hash

# AES Encryption using existing key
def aes_encrypt(message, aes_key):
    cipher = AES.new(aes_key, AES.MODE_CBC)
    ciphertext = cipher.iv + cipher.encrypt(pad(message.encode('utf-8'), AES.block_size))
    return base64.b64encode(ciphertext).decode('utf-8')

# RSA Encryption of AES Key using existing public key
def rsa_encrypt_key(aes_key, public_key_path):
    with open(public_key_path, "rb") as key_file:
        public_key = RSA.import_key(key_file.read())
    rsa_cipher = PKCS1_OAEP.new(public_key)
    encrypted_key = rsa_cipher.encrypt(aes_key)
    return base64.b64encode(encrypted_key).decode('utf-8')

# Embed data in LSB of audio
def embed_data(audio_path, output_path, binary_message, key):
    with wave.open(audio_path, 'rb') as audio:
        params = audio.getparams()
        frames = bytearray(audio.readframes(audio.getnframes()))

    random.seed(sum(key))  # Randomize based on the key
    positions = random.sample(range(len(frames)), len(binary_message))

    for i, bit in enumerate(binary_message):
        frames[positions[i]] = (frames[positions[i]] & 0xFE) | int(bit)

    with wave.open(output_path, 'wb') as stego_audio:
        stego_audio.setparams(params)
        stego_audio.writeframes(bytes(frames))

# Main process
if __name__ == "__main__":
    # Load AES Key
    with open(AES_KEY_PATH, "rb") as key_file:
        aes_key = key_file.read()

    # Read input message
    with open(INPUT_TEXT_PATH, "r") as msg_file:
        message = msg_file.read().strip()

    # Compute Hybrid Hash
    hash_value = hybrid_hash(message)
    with open(WORKING_DIR + "encrypted_data/hash_value.txt", "w") as hash_file:
        hash_file.write(hash_value)

    print(f"Hybrid Hash: {hash_value}")

    # Encrypt Message with AES
    encrypted_message = aes_encrypt(message, aes_key)

    # Encrypt AES Key using RSA Public Key
    encrypted_key = rsa_encrypt_key(aes_key, PUBLIC_KEY_PATH)

    # Concatenate Data
    combined_data = encrypted_message + "||" + encrypted_key + "||" + hash_value

    # Convert to binary
    binary_message = ''.join(format(ord(char), '08b') for char in combined_data)

    # Save combined data for verification
    with open(WORKING_DIR + "encrypted_data/combined_data.txt", "w") as combined_file:
        combined_file.write(combined_data)

    # Save binary message
    with open(WORKING_DIR + "encrypted_data/binary_message.txt", "w") as bin_file:
        bin_file.write(binary_message)

    # Embed into Audio
    if os.path.exists(INPUT_AUDIO_PATH):
        embed_data(INPUT_AUDIO_PATH, OUTPUT_AUDIO_PATH, binary_message, aes_key)
        print(f"✅ Data embedded into {OUTPUT_AUDIO_PATH}")
    else:
        print(f"❌ Input audio file not found at {INPUT_AUDIO_PATH}. Please upload it.")


Hybrid Hash: bb3b778f8c4bef065cc6487aac859f1ea5cebbee30a230b4ae5765eb0f60e359
✅ Data embedded into /content/sample_data/audio/stego_audio.wav


**for testing decoding of stego_audio **

In [ ]:
import wave
import os
import base64
import random
import hashlib
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Util.Padding import unpad

# Define working directory
WORKING_DIR = "/content/sample_data/"

# Paths to provided files
PRIVATE_KEY_PATH = WORKING_DIR + "keys/private_key.pem"
STEGO_AUDIO_PATH = WORKING_DIR + "audio/stego_audio.wav"
EXTRACTED_TEXT_PATH = WORKING_DIR + "encrypted_data/extracted_text.txt"

# Hybrid Hashing (MD5 → SHA-256) for verification
def hybrid_hash(message):
    md5_hash = hashlib.md5(message.encode('utf-8')).hexdigest()
    sha256_hash = hashlib.sha256(md5_hash.encode('utf-8')).hexdigest()
    return sha256_hash

# Extract binary data from LSB without AES key
def extract_data(audio_path, length=476):
    with wave.open(audio_path, 'rb') as audio:
        frames = bytearray(audio.readframes(audio.getnframes()))

    # Extract LSBs in sequential order (NO RANDOMIZATION since we don't have AES key yet)
    binary_message = ''.join(str((frames[i] & 1)) for i in range(length * 8))

    # Convert binary to text
    extracted_text = ''.join(chr(int(binary_message[i:i+8], 2)) for i in range(0, len(binary_message), 8))

    # Save extracted data for debugging
    with open(EXTRACTED_TEXT_PATH, "w") as extracted_file:
        extracted_file.write(extracted_text)

    return extracted_text

# RSA Decryption of AES Key
def rsa_decrypt_key(encrypted_key, private_key_path):
    with open(private_key_path, "rb") as key_file:
        private_key = RSA.import_key(key_file.read())
    rsa_cipher = PKCS1_OAEP.new(private_key)
    decrypted_key = rsa_cipher.decrypt(base64.b64decode(encrypted_key))
    return decrypted_key

# AES Decryption of Message
def aes_decrypt(ciphertext, aes_key):
    raw_data = base64.b64decode(ciphertext)
    iv = raw_data[:16]  # Extract IV
    cipher = AES.new(aes_key, AES.MODE_CBC, iv)
    decrypted_text = unpad(cipher.decrypt(raw_data[16:]), AES.block_size)
    return decrypted_text.decode('utf-8')

# Main decryption process
if __name__ == "__main__":
    # Extract data from stego audio
    extracted_text = extract_data(STEGO_AUDIO_PATH)

    # Split extracted data
    try:
        encrypted_message, encrypted_aes_key, received_hash = extracted_text.split("||")

        # Decrypt AES key using RSA
        decrypted_aes_key = rsa_decrypt_key(encrypted_aes_key, PRIVATE_KEY_PATH)

        # Decrypt message using AES key
        decrypted_message = aes_decrypt(encrypted_message, decrypted_aes_key)

        # Verify hash integrity
        computed_hash = hybrid_hash(decrypted_message)

        if computed_hash == received_hash:
            print("✅ Hash verification successful! Message is intact.")
        else:
            print("⚠️ Hash mismatch! Data may be tampered.")

        print("\n🔓 Decrypted Message:\n", decrypted_message)

    except ValueError:
        print("❌ Extraction error: Data may be corrupted or incomplete.")


❌ Extraction error: Data may be corrupted or incomplete.


In [ ]:
import cv2
import os

def replace_frames(video_path, stego_frames_folder, metadata_file, output_path):
    # Read metadata file for frame indices
    with open(metadata_file, 'r') as f:
        metadata_content = f.read().strip()

    if not metadata_content:
        print("❌ Error: Metadata file is empty.")
        return

    try:
        frame_indices = list(map(int, metadata_content.split(',')))
    except ValueError:
        print("❌ Error: Metadata file contains invalid data.")
        return

    # Open video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ Error: Unable to open video '{video_path}'.")
        return

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"🎥 Video Loaded: {total_frames} frames, {fps} FPS, {frame_width}x{frame_height} resolution.")

    # Define video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Replace frame if it's in metadata
        if frame_count in frame_indices:
            stego_frame_path = os.path.join(stego_frames_folder, f'stego_frame_{frame_count}.png')

            if os.path.exists(stego_frame_path):
                stego_frame = cv2.imread(stego_frame_path)

                if stego_frame is None:
                    print(f"⚠️ Warning: Could not read stego frame {stego_frame_path}. Skipping.")
                else:
                    stego_frame = cv2.resize(stego_frame, (frame_width, frame_height))
                    frame = stego_frame
            else:
                print(f"⚠️ Warning: Stego frame {stego_frame_path} not found. Using original frame.")

        out.write(frame)
        frame_count += 1

    # Release resources
    cap.release()
    out.release()

    print(f"✅ Modified video saved at: {output_path}")

# Example usage with corrected paths
replace_frames(
    video_path="/content/sample_data/muted_video.mp4",
    stego_frames_folder="/content/sample_data/stego_frames",  # Corrected folder path
    metadata_file="/content/sample_data/InfoOfSecretText/metadata.txt",  # Check metadata format
    output_path="/content/sample_data/modified_video.mp4"
)


🎥 Video Loaded: 1393 frames, 30 FPS, 576x1024 resolution.
✅ Modified video saved at: /content/sample_data/modified_video.mp4


In [ ]:
import subprocess

def merge_audio_ffmpeg(video_path, audio_path, output_path):
    try:
        command = [
            "ffmpeg", "-i", video_path, "-i", audio_path,
            "-c:v", "copy", "-c:a", "aac", "-b:a", "192k",  # Convert audio to AAC
            "-map", "0:v:0", "-map", "1:a:0",
            "-y", output_path
        ]
        subprocess.run(command, check=True)
        print(f"✅ Video merged successfully without compression: {output_path}")
    except subprocess.CalledProcessError as e:
        print(f"❌ FFmpeg Error: {e}")

merge_audio_ffmpeg(
    "/content/sample_data/modified_video.mp4",
    "/content/sample_data/audio/stego_audio.wav",
    "/content/sample_data/stego_video.mp4"
)


✅ Video merged successfully without compression: /content/sample_data/stego_video.mp4


In [ ]:
import os

# Paths
video_file_path = '/content/sample_data/stego_video.mp4'  # Replace with your video file path
audio_file_path = '/content/sample_data/audio/extracted_stego_audio.wav'  # Save extracted audio as WAV
muted_video_path = '/content/sample_data/muted_stego_video.mp4'  # Save muted video

# Extract audio from video
os.system(f'ffmpeg -i "{video_file_path}" -q:a 0 -map a "{audio_file_path}"')
print(f"Audio extracted and saved at: {audio_file_path}")

# Create muted video (video without audio)
os.system(f'ffmpeg -i "{video_file_path}" -c copy -an "{muted_video_path}"')
print(f"Muted video saved at: {muted_video_path}")


Audio extracted and saved at: /content/sample_data/audio/extracted_stego_audio.wav
Muted video saved at: /content/sample_data/muted_stego_video.mp4


In [ ]:
import wave
import os
import base64
import random
import hashlib
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Util.Padding import unpad

# Define working directory
WORKING_DIR = "/content/sample_data/"

# Hybrid Hash (MD5 → SHA-256) for integrity check
def hybrid_hash(message):
    md5_hash = hashlib.md5(message.encode('utf-8')).hexdigest()
    sha256_hash = hashlib.sha256(md5_hash.encode('utf-8')).hexdigest()
    return sha256_hash

# Extract binary data from LSB of audio
def extract_data(audio_path, key, length):
    with wave.open(audio_path, 'rb') as audio:
        frames = bytearray(audio.readframes(audio.getnframes()))

    random.seed(sum(key))  # Ensure same positions are used
    positions = random.sample(range(len(frames)), length)

    binary_message = ''.join(str(frames[pos] & 1) for pos in positions)

    return binary_message

# Convert binary to text
def binary_to_text(binary_string):
    chars = [binary_string[i:i+8] for i in range(0, len(binary_string), 8)]
    return ''.join(chr(int(char, 2)) for char in chars)

# RSA Decryption (AES Key)
def rsa_decrypt_key(encrypted_key, private_key):
    rsa_cipher = PKCS1_OAEP.new(RSA.import_key(private_key))
    return rsa_cipher.decrypt(base64.b64decode(encrypted_key))

# AES Decryption
def aes_decrypt(encrypted_message, aes_key):
    encrypted_message = base64.b64decode(encrypted_message)
    iv = encrypted_message[:AES.block_size]
    cipher = AES.new(aes_key, AES.MODE_CBC, iv)
    decrypted_text = unpad(cipher.decrypt(encrypted_message[AES.block_size:]), AES.block_size)
    return decrypted_text.decode('utf-8')

if __name__ == "__main__":
    stego_audio_path = WORKING_DIR + "audio/stego_audio.wav"
    private_key_path = WORKING_DIR + "keys/private_key.pem"

    # Load Private Key
    with open(private_key_path, "rb") as priv_key_file:
        private_key = priv_key_file.read()

    # Load length of binary message (assumed saved earlier)
    binary_message_path = os.path.join(WORKING_DIR, "encrypted_data", "binary_message.txt")
    with open(binary_message_path, "r") as f:
        binary_message = f.read()

    # Extract data from stego audio
    extracted_binary = extract_data(stego_audio_path, os.urandom(32), len(binary_message))
    extracted_text = binary_to_text(extracted_binary)

    # Split extracted data
    encrypted_message, encrypted_aes_key, received_hash = extracted_text.split("||")

    # Decrypt AES key using RSA
    decrypted_aes_key = rsa_decrypt_key(encrypted_aes_key, private_key)

    # Decrypt message using AES
    decrypted_message = aes_decrypt(encrypted_message, decrypted_aes_key)

    # Verify integrity
    computed_hash = hybrid_hash(decrypted_message)
    if computed_hash == received_hash:
        print("Integrity check passed!")
        output_text_path = os.path.join(WORKING_DIR, "decrypted_data", "info.txt")
        os.makedirs(os.path.dirname(output_text_path), exist_ok=True)
        with open(output_text_path, "w") as output_file:
            output_file.write(decrypted_message)
        print(f"Decrypted message saved to {output_text_path}")
    else:
        print("Integrity check failed! Data might have been tampered with.")


ValueError: not enough values to unpack (expected 3, got 1)

not required----------------------------------------------------------------------------------


In [ ]:
import cv2
import os
import base64
import hashlib
import numpy as np
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad

# Define directories
WORKING_DIR = "/content/sample_data/"
TEXT_SPLIT_DIR = os.path.join(WORKING_DIR, "split_texts/")
IMAGE_FOLDER = os.path.join(WORKING_DIR, "frames_folder/")  # Folder containing frames
OUTPUT_DIR = os.path.join(WORKING_DIR, "output_files/")
KEYS_DIR = os.path.join(WORKING_DIR, "keys/")

# Ensure necessary directories exist
os.makedirs(TEXT_SPLIT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Hybrid Hashing (MD5 -> SHA-256)
def hybrid_hash(message):
    md5_hash = hashlib.md5(message.encode('utf-8')).hexdigest()
    sha256_hash = hashlib.sha256(md5_hash.encode('utf-8')).hexdigest()
    return sha256_hash

# AES Encryption
def aes_encrypt(message, aes_key):
    iv = os.urandom(16)  # Generate a random IV
    cipher = AES.new(aes_key, AES.MODE_CBC, iv)
    ciphertext = cipher.encrypt(pad(message.encode('utf-8'), AES.block_size))
    encrypted_text = base64.b64encode(iv + ciphertext).decode('utf-8')
    return encrypted_text

# Convert text to binary
def text_to_binary(text):
    return ''.join(format(ord(char), '08b') for char in text)

# Embed data in LSB of image
def embed_data(image_path, output_path, binary_message):
    img = cv2.imread(image_path)
    if img is None:
        print(f"❌ Error: Image {image_path} not found!")
        return False

    height, width, channels = img.shape
    total_pixels = height * width * 3  # Total available bits for embedding

    if len(binary_message) > total_pixels:
        print(f"❌ Error: Message too large for {image_path}!")
        return False

    # Flatten image and embed data
    flat_img = img.flatten()
    for i in range(len(binary_message)):
        flat_img[i] = (flat_img[i] & 0xFE) | int(binary_message[i])  # Modify LSB

    # Reshape and save image
    stego_img = flat_img.reshape((height, width, channels))
    cv2.imwrite(output_path, stego_img)
    print(f"✅ Data embedded in {output_path}")
    return True

# Split secret text into 5 parts
def split_text_into_files(text, num_parts=5):
    chunk_size = len(text) // num_parts
    remainder = len(text) % num_parts
    split_texts = []
    start = 0

    for i in range(num_parts):
        end = start + chunk_size + (1 if i < remainder else 0)
        split_texts.append(text[start:end])
        start = end

    # Save split texts
    for i, part in enumerate(split_texts):
        file_path = os.path.join(TEXT_SPLIT_DIR, f"split_text_{i+1}.txt")
        with open(file_path, "w") as f:
            f.write(part)

    return split_texts

if __name__ == "__main__":
    input_text_path = os.path.join(WORKING_DIR, "secret_text.txt")
    aes_key_path = os.path.join(KEYS_DIR, "aes_key.txt")

    # Read the secret text
    with open(input_text_path, "r") as msg_file:
        full_message = msg_file.read().strip()

    # Step 1: Split text into 5 parts
    split_messages = split_text_into_files(full_message)

    # Step 2: Load AES Key (Decode if Base64 encoded)
    with open(aes_key_path, "r") as key_file:
        aes_key = base64.b64decode(key_file.read().strip())

    if len(aes_key) not in [16, 24, 32]:
        raise ValueError(f"❌ Error: Invalid AES key length ({len(aes_key)} bytes). Must be 16, 24, or 32 bytes.")

    # Step 3: Embed into images
    image_files = sorted(os.listdir(IMAGE_FOLDER))[:5]  # Get first 5 frames
    if len(image_files) < 5:
        print("❌ Error: Less than 5 images found in the image folder!")
    else:
        for i in range(5):
            text_part = split_messages[i]
            original_image_name = image_files[i]
            frame_number = ''.join(filter(str.isdigit, original_image_name))  # Extract frame number

            # Compute hash
            hash_value = hybrid_hash(text_part)
            with open(os.path.join(OUTPUT_DIR, f"hash_value_{frame_number}.txt"), "w") as hash_file:
                hash_file.write(hash_value)

            # Encrypt text with AES
            encrypted_message = aes_encrypt(text_part, aes_key)

            # Prepare data for embedding
            combined_data = encrypted_message + "||" + hash_value
            binary_message = text_to_binary(combined_data)

            # Embed into corresponding image
            image_path = os.path.join(IMAGE_FOLDER, original_image_name)
            output_image_path = os.path.join(OUTPUT_DIR, f"stego_frame_{frame_number}.png")
            embed_data(image_path, output_image_path, binary_message)

✅ Data embedded in /content/sample_data/output_files/stego_frame_0.png
✅ Data embedded in /content/sample_data/output_files/stego_frame_120.png
✅ Data embedded in /content/sample_data/output_files/stego_frame_140.png
✅ Data embedded in /content/sample_data/output_files/stego_frame_15.png
✅ Data embedded in /content/sample_data/output_files/stego_frame_20.png


In [ ]:
import os
import shutil
import ffmpeg

# Paths
WORKING_DIR = "/content/sample_data/"
FRAMES_DIR = os.path.join(WORKING_DIR, "frames_folder")  # Original extracted frames
STEGO_FRAMES_DIR = os.path.join(WORKING_DIR, "stego_frames")  # Stego frames directory
MODIFIED_FRAMES_DIR = os.path.join(WORKING_DIR, "modified_frames")  # Folder for modified frames
METADATA_FILE = "/content/sample_data/InfoOfSecretText/metadata.txt"
STEGO_VIDEO_PATH = os.path.join(WORKING_DIR, "stego_video.mp4")
STEGO_AUDIO_PATH = os.path.join(WORKING_DIR, "audio/stego_audio.wav")  # Provided stego audio file

# Ensure modified frames directory exists
if os.path.exists(MODIFIED_FRAMES_DIR):
    shutil.rmtree(MODIFIED_FRAMES_DIR)
os.makedirs(MODIFIED_FRAMES_DIR)

# Copy all frames from frames_folder to modified_frames
for frame in os.listdir(FRAMES_DIR):
    frame_path = os.path.join(FRAMES_DIR, frame)
    if os.path.isfile(frame_path):  # Ignore directories like .ipynb_checkpoints
        shutil.copy(frame_path, os.path.join(MODIFIED_FRAMES_DIR, frame))

print(f"✅ Copied all frames to {MODIFIED_FRAMES_DIR}")

# Read metadata file to get frame indexes
if os.path.exists(METADATA_FILE):
    with open(METADATA_FILE, "r") as f:
        metadata = f.read().strip()
    try:
        frame_indexes = sorted(set(int(i) for i in metadata.split(",") if i.strip().isdigit()))
    except ValueError:
        print("❌ Error: Invalid frame indexes found in metadata file.")
        exit()
else:
    print(f"❌ Metadata file not found at {METADATA_FILE}")
    exit()

print(f"ℹ️  Frames to replace: {frame_indexes}")

# Replace specific frames with stego frames
missing_frames = []
for index in frame_indexes:
    original_frame_name = f"frame_{index}.png"
    stego_frame_name = f"stego_frame_{index}.png"
    original_frame_path = os.path.join(MODIFIED_FRAMES_DIR, original_frame_name)
    stego_frame_path = os.path.join(STEGO_FRAMES_DIR, stego_frame_name)

    if os.path.exists(stego_frame_path):
        shutil.copy(stego_frame_path, original_frame_path)
        print(f"✅ Replaced {original_frame_name} with {stego_frame_name}")
    else:
        missing_frames.append(stego_frame_name)

if missing_frames:
    print(f"⚠️ Warning: The following stego frames were not found: {missing_frames}")

# Reconstruct video from modified frames
TEMP_VIDEO_PATH = os.path.join(WORKING_DIR, "temp_video.mp4")
(
    ffmpeg
    .input(os.path.join(MODIFIED_FRAMES_DIR, "frame_%d.png"), framerate=30)
    .output(TEMP_VIDEO_PATH, pix_fmt="yuv420p", vcodec="libx264", crf=23)
    .run(overwrite_output=True)
)

print(f"✅ Temporary video created: {TEMP_VIDEO_PATH}")

# Merge stego audio with the new video
if os.path.exists(STEGO_AUDIO_PATH):
    video_input = ffmpeg.input(TEMP_VIDEO_PATH)
    audio_input = ffmpeg.input(STEGO_AUDIO_PATH)

    ffmpeg.output(
        video_input, audio_input, STEGO_VIDEO_PATH,
        vcodec="libx264", acodec="aac", strict="experimental", shortest=True
    ).run(overwrite_output=True)

    print(f"✅ Stego video with audio saved at: {STEGO_VIDEO_PATH}")
else:
    print(f"❌ Stego audio file not found at {STEGO_AUDIO_PATH}")



In [ ]:
import subprocess
subprocess.run(["ffmpeg", "-version"])


CompletedProcess(args=['ffmpeg', '-version'], returncode=0)

In [ ]:
import os
frames = os.listdir("/content/sample_data/modified_frames/")
print(frames[:10])  # Show first 10 frames


['frame_175.png', 'frame_160.png', 'frame_180.png', 'frame_185.png', 'metadata.txt', 'predictions.csv', 'frame_190.png']


stego image for one image

In [ ]:
import cv2
import os
import base64
import hashlib
import numpy as np
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad

# Define working and output directories
WORKING_DIR = "/content/sample_data/"
OUTPUT_DIR = "/content/sample_data/output_file/"

# Ensure the output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Hybrid Hashing (MD5 → SHA-256)
def hybrid_hash(message):
    md5_hash = hashlib.md5(message.encode('utf-8')).hexdigest()
    sha256_hash = hashlib.sha256(md5_hash.encode('utf-8')).hexdigest()
    return sha256_hash

# AES Encryption
def aes_encrypt(message, aes_key):
    cipher = AES.new(aes_key, AES.MODE_CBC)
    ciphertext = cipher.iv + cipher.encrypt(pad(message.encode('utf-8'), AES.block_size))

    encrypted_text = base64.b64encode(ciphertext).decode('utf-8')

    # Save encrypted text
    with open(OUTPUT_DIR + "encrypted_text.txt", "w") as enc_file:
        enc_file.write(encrypted_text)

    return encrypted_text

# Convert text to binary
def text_to_binary(text):
    return ''.join(format(ord(char), '08b') for char in text)

# Embed data in LSB of image
def embed_data(image_path, output_path, binary_message):
    img = cv2.imread(image_path)
    if img is None:
        print("❌ Error: Image not found or could not be loaded!")
        return

    height, width, channels = img.shape
    total_pixels = height * width * 3  # Total available bits for embedding

    if len(binary_message) > total_pixels:
        print("❌ Error: Message too large to fit in the image!")
        return

    # Flatten image and embed data
    flat_img = img.flatten()
    for i in range(len(binary_message)):
        flat_img[i] = (flat_img[i] & 0xFE) | int(binary_message[i])  # Replace LSB

    # Reshape and save image
    stego_img = flat_img.reshape((height, width, channels))
    cv2.imwrite(output_path, stego_img)
    print(f"✅ Data successfully embedded into {output_path}")

if __name__ == "__main__":
    input_image_path = WORKING_DIR + "frames_folder/frame_160.png"  # Input image for embedding
    output_image_path = OUTPUT_DIR + "output_stego_image.png"
    input_text_path = WORKING_DIR + "secret_text.txt"
    aes_key_path = WORKING_DIR + "aes_key.txt"  # AES key already exists

    # Read input message from file
    with open(input_text_path, "r") as msg_file:
        message = msg_file.read().strip()

    # Step 1: Compute Hybrid Hash (MD5 → SHA-256)
    hash_value = hybrid_hash(message)
    with open(OUTPUT_DIR + "hash_value.txt", "w") as hash_file:
        hash_file.write(hash_value)
    print(f"🔒 Hybrid Hash: {hash_value}")

    # Step 2: Load Existing AES Key
    with open(aes_key_path, "rb") as key_file:
        aes_key = key_file.read()

    # Step 3: Encrypt Message with AES
    encrypted_message = aes_encrypt(message, aes_key)

    # Step 4: Prepare data for embedding (AES encrypted text + hash)
    combined_data = encrypted_message + "||" + hash_value
    with open(OUTPUT_DIR + "combined_data.txt", "w") as combined_file:
        combined_file.write(combined_data)

    # Convert combined data to binary
    binary_message = text_to_binary(combined_data)
    print(f"📡 Binary message length: {len(binary_message)} bits")

    # Step 5: Embed into Image
    if os.path.exists(input_image_path):
        embed_data(input_image_path, output_image_path, binary_message)
    else:
        print(f"❌ Error: Input image file not found at {input_image_path}. Please provide a valid image.")


🔒 Hybrid Hash: 5a749373887c0d5d6a2ed44e03b7fb2804c7d39fea31d583909f9aa386516c84
📡 Binary message length: 32784 bits
✅ Data successfully embedded into /content/sample_data/output_file/output_stego_image.png


In [ ]:
import cv2
import os
import base64
import hashlib
import numpy as np
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad

# Define working and output directories
WORKING_DIR = "/content/sample_data/"
OUTPUT_DIR = "/content/sample_data/output_file/"

# Ensure the output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Function to extract binary data from image
def extract_data(image_path, data_length):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Error: Image not found or could not be loaded!")

    flat_img = img.flatten()
    binary_message = ''.join(str(flat_img[i] & 1) for i in range(data_length))
    extracted_text = ''.join(chr(int(binary_message[i:i+8], 2)) for i in range(0, len(binary_message), 8))

    with open(OUTPUT_DIR + "extracted_text.txt", "w") as f:
        f.write(extracted_text)

    return extracted_text

# AES Decryption
def aes_decrypt(ciphertext, aes_key):
    raw_data = base64.b64decode(ciphertext)
    iv, ciphertext = raw_data[:AES.block_size], raw_data[AES.block_size:]
    cipher = AES.new(aes_key, AES.MODE_CBC, iv)
    decrypted_data = unpad(cipher.decrypt(ciphertext), AES.block_size).decode('utf-8')

    with open(OUTPUT_DIR + "decrypted_text.txt", "w") as f:
        f.write(decrypted_data)

    return decrypted_data

# Compute Hybrid Hash (MD5 → SHA-256)
def compute_hybrid_hash(plaintext):
    md5_hash = hashlib.md5(plaintext.encode()).hexdigest()
    sha256_hash = hashlib.sha256(md5_hash.encode()).hexdigest()

    with open(OUTPUT_DIR + "recomputed_hash.txt", "w") as f:
        f.write(sha256_hash)

    return sha256_hash

# Verify Integrity
def verify_hash(extracted_hash, recomputed_hash):
    with open(OUTPUT_DIR + "verification_result.txt", "w") as f:
        if extracted_hash == recomputed_hash:
            f.write("Success: The message is verified and intact.\n")
            return True
        else:
            f.write("Error: The message is corrupted or tampered with.\n")
            return False

if __name__ == "__main__":
    stego_image_path = "/content/sample_data/output_file/output_stego_image.png"
    aes_key_path = WORKING_DIR + "aes_key.txt"  # Load existing AES key

    # Read AES key
    with open(aes_key_path, "rb") as key_file:
        aes_key = key_file.read()

    # Read the original data length for extraction
    with open(OUTPUT_DIR + "combined_data.txt", "r") as f:
        combined_data = f.read().strip()

    binary_message_length = len(combined_data) * 8  # Convert to bit length

    # Step 1: Extract data from image
    extracted_text = extract_data(stego_image_path, binary_message_length)

    # Step 2: Separate extracted ciphertext and hash
    extracted_ciphertext, extracted_hash = extracted_text.split('||', 1)

    with open(OUTPUT_DIR + "extracted_ciphertext.txt", "w") as f:
        f.write(extracted_ciphertext)
    with open(OUTPUT_DIR + "extracted_hash.txt", "w") as f:
        f.write(extracted_hash)

    # Step 3: Decrypt AES Ciphertext
    decrypted_message = aes_decrypt(extracted_ciphertext, aes_key)

    # Step 4: Compute Hash of Decrypted Message
    recomputed_hash = compute_hybrid_hash(decrypted_message)

    # Step 5: Verify Integrity
    verification_status = verify_hash(extracted_hash, recomputed_hash)

    print("Decoding complete. Verification status saved in verification_result.txt")


Decoding complete. Verification status saved in verification_result.txt


In [ ]:
import wave
import os
import base64
import random
import hashlib
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Util.Padding import unpad

# Define working directory
WORKING_DIR = "/content/sample_data/"

# Paths to provided files
PRIVATE_KEY_PATH = WORKING_DIR + "keys/private_key.pem"
STEGO_AUDIO_PATH = WORKING_DIR + "audio/stego_audio.wav"
EXTRACTED_TEXT_PATH = WORKING_DIR + "encrypted_data/extracted_text.txt"

# Hybrid Hashing (MD5 → SHA-256) for verification
def hybrid_hash(message):
    md5_hash = hashlib.md5(message.encode('utf-8')).hexdigest()
    sha256_hash = hashlib.sha256(md5_hash.encode('utf-8')).hexdigest()
    return sha256_hash

# Extract binary data from LSB (476 characters)
def extract_data(audio_path, length=476):
    with wave.open(audio_path, 'rb') as audio:
        frames = bytearray(audio.readframes(audio.getnframes()))

    # Extract LSBs from a fixed number of bits
    binary_message = ''.join(str((frames[i] & 1)) for i in range(length * 8))

    # Convert binary to text
    extracted_text = ''.join(chr(int(binary_message[i:i+8], 2)) for i in range(0, len(binary_message), 8))

    # Ensure exactly 476 characters
    extracted_text = extracted_text[:476]

    # Save extracted text for verification
    with open(EXTRACTED_TEXT_PATH, "w") as extracted_file:
        extracted_file.write(extracted_text)

    return extracted_text

# RSA Decryption of AES Key
def rsa_decrypt_key(encrypted_key, private_key_path):
    with open(private_key_path, "rb") as key_file:
        private_key = RSA.import_key(key_file.read())
    rsa_cipher = PKCS1_OAEP.new(private_key)
    decrypted_key = rsa_cipher.decrypt(base64.b64decode(encrypted_key))
    return decrypted_key

# AES Decryption of Message
def aes_decrypt(ciphertext, aes_key):
    raw_data = base64.b64decode(ciphertext)
    iv = raw_data[:16]  # Extract IV
    cipher = AES.new(aes_key, AES.MODE_CBC, iv)
    decrypted_text = unpad(cipher.decrypt(raw_data[16:]), AES.block_size)
    return decrypted_text.decode('utf-8')

# Main decryption process
if __name__ == "__main__":
    # Extract data from stego audio
    extracted_text = extract_data(STEGO_AUDIO_PATH, length=476)

    # Split extracted data
    try:
        encrypted_message, encrypted_aes_key, received_hash = extracted_text.split("||")

        # Decrypt AES key using RSA
        decrypted_aes_key = rsa_decrypt_key(encrypted_aes_key, PRIVATE_KEY_PATH)

        # Decrypt message using AES key
        decrypted_message = aes_decrypt(encrypted_message, decrypted_aes_key)

        # Verify hash integrity
        computed_hash = hybrid_hash(decrypted_message)

        if computed_hash == received_hash:
            print("✅ Hash verification successful! Message is intact.")
        else:
            print("⚠️ Hash mismatch! Data may be tampered.")

        print("\n🔓 Decrypted Message:\n", decrypted_message)

    except ValueError:
        print("❌ Extraction error: Data may be corrupted or incomplete.")


❌ Extraction error: Data may be corrupted or incomplete.


In [ ]:
import os

def get_text_length(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            return len(text)
    else:
        return "File not found!"

# Example usage
file_path = "/content/sample_data/encrypted_data/combined_data.txt"  # Replace with your actual file path
length = get_text_length(file_path)
print(f"Length of the text file: {length} characters")

s="kzx4L0R58USxbgVQQB9qAUBU4UbSimaeayhNrncdv63lK+rbfjZOhVOQMS+g3jFM0LQD1HrlxdFuVvnXBf9Szr/8LDr8PEQHCmrYA0aQrNjL4aJS2dcW8LUJB4CrvolYpLt0jZWFKWza4pI4pix3spHOOs2ZiA5lPt7L96WoTQNPCRzMRu194Ex6ID4/QZYxYYMG1eoeTyYFI3F9nL1/JQy6qW1CfP2/WtYrZcXDPw41o0vYyf56AO7TBtC0M6IMmtJe8t23R/evzC3bi3afMPyiMxIkl5uvetpo0KgfRgOLv0vfCOSGUgt5ImB09dN9xn++ceEGczlcz4D8jvXYlUIpJ1yoj1GDB72s84UX8L5/FOmwsusuMnn9sAZjQoAo/iQM+8cp8CJIWPe2w8LqhscjIhou2DXfEgdKxuiKgxQuBMIixR2mh2xIYqUNZYKExdN6JbOJHyjGm/IgwDSEUhMJiTvUmzhxbBndHiQ8PCHLrxHK3LllB+a3YtecoZ5UVrhbvWrm5SjeB6vx4bx7qjDcHwDS419bb/1PbzfJgAqooaYwT572TUAkdprXTzEDLCylZQakCp7gEXSpErITu5ha9hnzVDQ3G0/MypgWZYSDcWWnjveGiSXmeSNRiTrSu0t/DGvS7xhXynt7f11rMabBx154wYYNGlEW5MJs/mR3JE08j8ILiXN8ywTq1wH+Rvyj5BaNfTKuyH6nFA6/56RRJEzSXZAOSEjYcNoZViOBhg5+DOoCTxtkwo/KdA0YRAtLqqX2vR1/2lKMqhfFVD10sUxbrXGW5zaFrZ/lFVRd2A3uW8IV5AeIGkl+VQ2qL9ojrtaDzhtixjohtgVzX7PQ7zuGEKRPgg2pvKaIsBqVtiwDOWhnNH1H6udqsoVLYu8Ctpl+0/tt9Qt8r/u+XwRYnc6nmODDY86TaTeIC/X+gldkMtH1j7LUyeNW0gLgzGCguEen8NHotRamGA6pNfZRYyXoTED2wGdjeBqTcReVN4qt32g9zHJnNY5lPFVzi8YoSMkR9YV2lt9YnjMYhe0rgLzv9SDiR9TxFoO9grwk1evP0Cl/0ntgx5CymIy6JEEFTYfumZqn4olHa52ES70a3j6t48TiXb8HDGQxR3bAQ8X/Ixj39o8D3oQJfQdnc8JfrKobVvORBaj7f75a9QrtGZJvAlEytvPMkwnF9DXwnvkD4d097YfAZaYAOVgMinlXSgc2ikjgin+f58j6zrtEmPF7IBYZBKIzAtKsUMq6UQ0E8LXhWYPMOqc2QgNnQ/V8DbVdizVzOKYSXiN2netkCzXTEdefqSv2Ob0czIg/H4K6n1PoNyOxHFH9n55fkRCR68ai7nm5XDIs2vQBxga40Yh8exT3wyDojwJIfGtLUFwKcX0G65Z9/JtD2BvrrznNiFg9+Hjq+FV8LOT9EkPspN5p6KN8JPH3qvKEVfpyD92+vbPpKTRXNs3ydZ4w58dTGm03SmdTkNlYDDMwu5jhFnLQn4Zvdrp2fXGZ99cOko8pTEKC6jMb3L+KnSdmSN7lY3H0Pcl8kBpm2D2lDlD/lGHi3JeaB7KAmd9YOAz8EA56lPfGkcJGOXcxRoSo4vS+xFWLn8z1X1h85FljETjRUbtwhISoDCBNyWpkAsbZpt6e3IqxCrjkRMVXIuzv5FI20xafOWFrDpu44n7lMRAhipvjwHOUyGhE4R4GrkJxD0MKs3NoOZNx1lGYSi/isZ2Fx3CqoulhS5nnFNc9nXiUJMjY4qQnUBA6TKp9LY+DW1LSRQNLSsFovRiyofqfR43E2WHqYj1yBhkWHHL7XLHcOoMWeV/5PCsC23ihkNluAAUFDtxbzf+SLiDLACOO76rPwIVZHh1EH/IAxX4m0iRQKDAtIADt0bnBP0FOxpAxEcUZaASrwDCni6SX4C6oIiVWdQ6Nl6SPFDo9XuSPkCRziF24FTeD42ULf0C05Gilc82VolgN2xnXKQYtoL0WsxxKqck/z7pm/sfo71pLeoI7oukTSV1UmH+l+ySk5jGdNhXngeBmbOvKtkjlQBKvLUy3EmGCzYGmXtu5Wdzxk8K6orDzOv/WN3rQYTp6xOdjZ3SRd+jd+amZy0JicX9zc+0L7Qql+Be02KQY9NfOnl0FCu/XwXbY4Q+55ogWncxaFpXWZoY3UQw0wvsq7UdWogt6RKuaS37MHyJV4BOB3urV9n2Bd15QPbovi6kdv/m5iJJkESEuBb9/AE3X0voBNrbORrCmh7m0ngWrzURiYydzGT9g2NCVE6PCf4ddhOp3DJFNXMjWJxXFckAc1jNnhgnkYl/rU+uSaKlU/VBn2X7Ad/7dEdc7BJAb254EP9fmRMqW19J4FhTl8vuw1yMGqUG93J0WzEzMT2J1EJwIAe+G7i2xlDqoBUwlPaQse4uAjBwviPNU59nnkISddZfs3vr2i+s6cRVoFtDQfXsshQCFnZeRBu96yEy4xF/Y7WdddXJOYeg6LO+x1Ytt5Fx3VnW6oDcQieBAIjpRcBo1dqJAqMiKbqdd0cfkN9sCgCFH+zkQHS5KKlam/DyqC2S7jI6KdazDkoKqF5FEUjUn8JJCywS2UWSt7WR6HSgVmmNAn49Eb13wmgRjbpLeImK3JHWo2yVLuJZffb0j7/BciHKQ/K5BUJu2YnRnJx7vEQaUi0aPtoZyBBXmvjfJKzP8ZPDd0RK2mZREspLhbtYE5nuyyIANxAIJc+Fgdz/MKCbpUlPWdIu9UZjx5ZpYZrL8Ia94U/4XMJ3LuXqqum9jKBYBHQZKWx0w8lVHm9m1HOp/0/t4TitOWV/+FzGDPwAmdWBZPr6GSqtpMxv0FuoCPBC2BE+b4TdpnVmowwb3tn45cja+scmt4qmKtDvi9o7t9Ki/7h3bOsN221Bzenj39Z8PFivKb/coCothHpz6bcHHyhPpxi0dIUr6gMN2+vbahv3h9Hv/giRTe5xo6XlBNRouHbfu38WP8Px5JXDXmPWenPq0JNqXvXDtiK0slQHB2vqJfx5wprzyQFVAUQ019bV5GgQ1Rjal3PZhHFFSK/a5d/hn51lpdOpgxU4DF3+ZwpeKgo6+XA2WBvIGiIANSg63nYf0O5VxWrT3exjV32Yt0ehU6uSWnZ/5URWkJEbAiBYCLaejALJAA/42AwNeqaWiIdmcc4JcmiNEgyhtyAUiFNuh4Dk/XT5n91eOR1rK3aIdU//NHAhUX/mDC1EmNjT2s3ZlpUMcU/kv647SyL/vEC4kUlPkPyy3t01SAcGwHbyPfT2uRuI3DOutNhocB5NEHZk4aYf98N5RZ9ZdElNSQisnKyjO/CVBBIa/BUI7f2ZwLmX8hWwY/I6VXbxr2hFIwbS0zedGyWS/011/di4TVo1KJcPS0I2nw4byjcRZqrzclln/gy/UrfrKg5T/n7LdryxeraHK/OBXUUnZI7Kqt7wGmMPlPDuZMLPxIDRpcoH6pAwt97ZUfra/zZIodK/gAHE1Kk/5Unp81eQ/Z5ZSZSR+hh1iiEBq8hEVayaXNu3Cagcr38TvS8NhF+RClifmPUeERLH69F/bmUVmkBuGBhbuFEdXsEZ5beyr+90ONCVtfQJOfRrJdR8nWpd1Wlwima85snFACYBIiXGIa3kE5uYsIhqyHNI2/pcx0P/TzZy83EUzEHrquyYyUictXJBcS7ONY/kwdcZ0Pvjbi4+ERgpi8oXBW9XZHXErlamsHY8sV4eyvN2dVmZs4Q9/sIjopBRkl2jnF8SUTg16+W74tFvCjQK8T4EcuDYZMtjIDQk/8wyOT423E4OzBodNsjgJP4GJr36PxQPUdBc6ezYExLRIdeqm3pGqx4ULyKHBhAfwfZ24g54W6Gaa+n59Ct08i+/USIqbzCwf8fvlaiG8N7Dfkc8+yPFACPOSY1lChTnzIF3uyUSRHIJML9jXfrfoQmwtujnXoihfixPPoWtC7fh24C+ESagGNKexjUUOUSYxHHpi5tMx2xqzoc3KQzAgmcGfKjoMxfCGGIFHWEtlQkoB7xGyCIyTcp+BryOn"
print(len(s))
s1="kzx4L0R58USxbgVQQB9qAUBU4UbSimaeayhNrncdv63lK+rbfjZOhVOQMS+g3jFM0LQD1HrlxdFuVvnXBf9Szr/8LDr8PEQHCmrYA0aQrNjL4aJS2dcW8LUJB4CrvolYpLt0jZWFKWza4pI4pix3spHOOs2ZiA5lPt7L96WoTQNPCRzMRu194Ex6ID4/QZYxYYMG1eoeTyYFI3F9nL1/JQy6qW1CfP2/WtYrZcXDPw41o0vYyf56AO7TBtC0M6IMmtJe8t23R/evzC3bi3afMPyiMxIkl5uvetpo0KgfRgOLv0vfCOSGUgt5ImB09dN9xn++ceEGczlcz4D8jvXYlUIpJ1yoj1GDB72s84UX8L5/FOmwsusuMnn9sAZjQoAo/iQM+8cp8CJIWPe2w8LqhscjIhou2DXfEgdKxuiKgxQuBMIixR2mh2xIYqUNZYKExdN6JbOJHyjGm/IgwDSEUhMJiTvUmzhxbBndHiQ8PCHLrxHK3LllB+a3YtecoZ5UVrhbvWrm5SjeB6vx4bx7qjDcHwDS419bb/1PbzfJgAqooaYwT572TUAkdprXTzEDLCylZQakCp7gEXSpErITu5ha9hnzVDQ3G0/MypgWZYSDcWWnjveGiSXmeSNRiTrSu0t/DGvS7xhXynt7f11rMabBx154wYYNGlEW5MJs/mR3JE08j8ILiXN8ywTq1wH+Rvyj5BaNfTKuyH6nFA6/56RRJEzSXZAOSEjYcNoZViOBhg5+DOoCTxtkwo/KdA0YRAtLqqX2vR1/2lKMqhfFVD10sUxbrXGW5zaFrZ/lFVRd2A3uW8IV5AeIGkl+VQ2qL9ojrtaDzhtixjohtgVzX7PQ7zuGEKRPgg2pvKaIsBqVtiwDOWhnNH1H6udqsoVLYu8Ctpl+0/tt9Qt8r/u+XwRYnc6nmODDY86TaTeIC/X+gldkMtH1j7LUyeNW0gLgzGCguEen8NHotRamGA6pNfZRYyXoTED2wGdjeBqTcReVN4qt32g9zHJnNY5lPFVzi8YoSMkR9YV2lt9YnjMYhe0rgLzv9SDiR9TxFoO9grwk1evP0Cl/0ntgx5CymIy6JEEFTYfumZqn4olHa52ES70a3j6t48TiXb8HDGQxR3bAQ8X/Ixj39o8D3oQJfQdnc8JfrKobVvORBaj7f75a9QrtGZJvAlEytvPMkwnF9DXwnvkD4d097YfAZaYAOVgMinlXSgc2ikjgin+f58j6zrtEmPF7IBYZBKIzAtKsUMq6UQ0E8LXhWYPMOqc2QgNnQ/V8DbVdizVzOKYSXiN2netkCzXTEdefqSv2Ob0czIg/H4K6n1PoNyOxHFH9n55fkRCR68ai7nm5XDIs2vQBxga40Yh8exT3wyDojwJIfGtLUFwKcX0G65Z9/JtD2BvrrznNiFg9+Hjq+FV8LOT9EkPspN5p6KN8JPH3qvKEVfpyD92+vbPpKTRXNs3ydZ4w58dTGm03SmdTkNlYDDMwu5jhFnLQn4Zvdrp2fXGZ99cOko8pTEKC6jMb3L+KnSdmSN7lY3H0Pcl8kBpm2D2lDlD/lGHi3JeaB7KAmd9YOAz8EA56lPfGkcJGOXcxRoSo4vS+xFWLn8z1X1h85FljETjRUbtwhISoDCBNyWpkAsbZpt6e3IqxCrjkRMVXIuzv5FI20xafOWFrDpu44n7lMRAhipvjwHOUyGhE4R4GrkJxD0MKs3NoOZNx1lGYSi/isZ2Fx3CqoulhS5nnFNc9nXiUJMjY4qQnUBA6TKp9LY+DW1LSRQNLSsFovRiyofqfR43E2WHqYj1yBhkWHHL7XLHcOoMWeV/5PCsC23ihkNluAAUFDtxbzf+SLiDLACOO76rPwIVZHh1EH/IAxX4m0iRQKDAtIADt0bnBP0FOxpAxEcUZaASrwDCni6SX4C6oIiVWdQ6Nl6SPFDo9XuSPkCRziF24FTeD42ULf0C05Gilc82VolgN2xnXKQYtoL0WsxxKqck/z7pm/sfo71pLeoI7oukTSV1UmH+l+ySk5jGdNhXngeBmbOvKtkjlQBKvLUy3EmGCzYGmXtu5Wdzxk8K6orDzOv/WN3rQYTp6xOdjZ3SRd+jd+amZy0JicX9zc+0L7Qql+Be02KQY9NfOnl0FCu/XwXbY4Q+55ogWncxaFpXWZoY3UQw0wvsq7UdWogt6RKuaS37MHyJV4BOB3urV9n2Bd15QPbovi6kdv/m5iJJkESEuBb9/AE3X0voBNrbORrCmh7m0ngWrzURiYydzGT9g2NCVE6PCf4ddhOp3DJFNXMjWJxXFckAc1jNnhgnkYl/rU+uSaKlU/VBn2X7Ad/7dEdc7BJAb254EP9fmRMqW19J4FhTl8vuw1yMGqUG93J0WzEzMT2J1EJwIAe+G7i2xlDqoBUwlPaQse4uAjBwviPNU59nnkISddZfs3vr2i+s6cRVoFtDQfXsshQCFnZeRBu96yEy4xF/Y7WdddXJOYeg6LO+x1Ytt5Fx3VnW6oDcQieBAIjpRcBo1dqJAqMiKbqdd0cfkN9sCgCFH+zkQHS5KKlam/DyqC2S7jI6KdazDkoKqF5FEUjUn8JJCywS2UWSt7WR6HSgVmmNAn49Eb13wmgRjbpLeImK3JHWo2yVLuJZffb0j7/BciHKQ/K5BUJu2YnRnJx7vEQaUi0aPtoZyBBXmvjfJKzP8ZPDd0RK2mZREspLhbtYE5nuyyIANxAIJc+Fgdz/MKCbpUlPWdIu9UZjx5ZpYZrL8Ia94U/4XMJ3LuXqqum9jKBYBHQZKWx0w8lVHm9m1HOp/0/t4TitOWV/+FzGDPwAmdWBZPr6GSqtpMxv0FuoCPBC2BE+b4TdpnVmowwb3tn45cja+scmt4qmKtDvi9o7t9Ki/7h3bOsN221Bzenj39Z8PFivKb/coCothHpz6bcHHyhPpxi0dIUr6gMN2+vbahv3h9Hv/giRTe5xo6XlBNRouHbfu38WP8Px5JXDXmPWenPq0JNqXvXDtiK0slQHB2vqJfx5wprzyQFVAUQ019bV5GgQ1Rjal3PZhHFFSK/a5d/hn51lpdOpgxU4DF3+ZwpeKgo6+XA2WBvIGiIANSg63nYf0O5VxWrT3exjV32Yt0ehU6uSWnZ/5URWkJEbAiBYCLaejALJAA/42AwNeqaWiIdmcc4JcmiNEgyhtyAUiFNuh4Dk/XT5n91eOR1rK3aIdU//NHAhUX/mDC1EmNjT2s3ZlpUMcU/kv647SyL/vEC4kUlPkPyy3t01SAcGwHbyPfT2uRuI3DOutNhocB5NEHZk4aYf98N5RZ9ZdElNSQisnKyjO/CVBBIa/BUI7f2ZwLmX8hWwY/I6VXbxr2hFIwbS0zedGyWS/011/di4TVo1KJcPS0I2nw4byjcRZqrzclln/gy/UrfrKg5T/n7LdryxeraHK/OBXUUnZI7Kqt7wGmMPlPDuZMLPxIDRpcoH6pAwt97ZUfra/zZIodK/gAHE1Kk/5Unp81eQ/Z5ZSZSR+hh1iiEBq8hEVayaXNu3Cagcr38TvS8NhF+RClifmPUeERLH69F/bmUVmkBuGBhbuFEdXsEZ5beyr+90ONCVtfQJOfRrJdR8nWpd1Wlwima85snFACYBIiXGIa3kE5uYsIhqyHNI2/pcx0P/TzZy83EUzEHrquyYyUictXJBcS7ONY/kwdcZ0Pvjbi4+ERgpi8oXBW9XZHXErlamsHY8sV4eyvN2dVmZs4Q9/sIjopBRkl2jnF8SUTg16+W74tFvCjQK8T4EcuDYZMtjIDQk/8wyOT423E4OzBodNsjgJP4GJr36PxQPUdBc6ezYExLRIdeqm3pGqx4ULyKHBhAfwfZ24g54W6Gaa+n59Ct08i+/USIqbzCwf8fvlaiG8N7Dfkc8+yPFACPOSY1lChTnzIF3uyUSRHIJML9jXfrfoQmwtujnXoihfixPPoWtC7fh24C+ESagGNKexjUUOUSYxHHpi5tMx2xqzoc3KQzAgmcGfKjoMxfCGGIFHWEtlQkoB7xGyCIyTcp+BryOn"

Length of the text file: 476 characters
4032
